In [1]:
import psycopg2
import pandas as pd
import sqlalchemy

In [14]:
conn = psycopg2.connect(host="127.0.0.1", port="5432", database="Adventureworks", 
                            user="postgres", password="postgres")
cur = conn.cursor()

### смотрим, что за данные

In [3]:
query = f'''
         select * from sales.customer
        where territoryid =8
'''
df = pd.read_sql_query(query, conn)
df.head()

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,customerid,personid,storeid,territoryid,rowguid,modifieddate
0,14,NaN,1020.0,8,2f96bedc-723d-468f-834b-b2b8ae79c849,2014-09-12 11:15:07.263
1,32,NaN,1008.0,8,b9a28813-542a-4fbf-ae79-bad64a5e133b,2014-09-12 11:15:07.263
2,50,NaN,996.0,8,0a3e846a-2dcc-4b6e-8ab3-968b5ab859f4,2014-09-12 11:15:07.263
3,68,NaN,532.0,8,2138def5-80eb-4d95-b340-706bcd60adda,2014-09-12 11:15:07.263
4,86,NaN,636.0,8,2a8b2702-4644-4002-90e4-99a8adba5261,2014-09-12 11:15:07.263


### Есть два пути. Один из них - агрегировать

In [4]:
query = f'''
         SELECT array_agg(customerid::text)
        FROM sales.customer
        where territoryid =4
'''
df = pd.read_sql_query(query, conn)
df.head()

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,array_agg
0,"[3, 4, 5, 6, 21, 23, 24, 25, 40, 41, 42, 59, 6..."


### Другой - использовать конструктор массива:
### где customerid собирается по territoryid = 8, а массив по where territoryid =10
### это только рекордсет, нельзя обратиться по имени acc_array или cus. используется в констуркции insert into select
### SELECT customerid, array(

In [8]:
query = f'''
         SELECT customerid, array(
        SELECT rowguid ::TEXT
         FROM sales.customer
		where territoryid =10
        )
           AS acc_array 
        FROM sales.customer as cus
        where territoryid = 8;
'''
df = pd.read_sql_query(query, conn)
df.head()

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,customerid,acc_array
0,14,"[c9381589-d31c-4efe-8978-8d3449eb1f0f, 9edda79..."
1,32,"[c9381589-d31c-4efe-8978-8d3449eb1f0f, 9edda79..."
2,50,"[c9381589-d31c-4efe-8978-8d3449eb1f0f, 9edda79..."
3,68,"[c9381589-d31c-4efe-8978-8d3449eb1f0f, 9edda79..."
4,86,"[c9381589-d31c-4efe-8978-8d3449eb1f0f, 9edda79..."


### выборки из таблицы, где несколько полей = массив


In [9]:
cur.execute('''
           CREATE TABLE grades (
                student_id int,
                email text[][],
                test_scores int[]
            );
            ''')

cur.execute('''
            INSERT INTO grades
            VALUES(1, '{ {"work", "work1@gmail.com"}, {"home", "home1@gmail.com"} }',
            '{92, 58,98, 100}');
            ''')

cur.execute('''
            SELECT
            email[1][1] as type, -- обращаться можно по индексу массива
            email[1][2] as address,
            test_scores[1]
            FROM
            grades;
            ''')


results = cur.fetchall()
pd.DataFrame(results, columns=['type', 'address', 'test_scores'])

,type,address,test_scores
0,work,work1@gmail.com,92


### создать свой сложный тип и его возвращать


In [15]:
cur.execute('''
            CREATE TYPE compozit AS (f1 text, f2 text);
            ''')

cur.execute('''
            CREATE FUNCTION getCompozit() RETURNS SETOF compozit AS $$
            SELECT customerid, rowguid FROM sales.customer
            $$ LANGUAGE SQL;
            ''')

cur.execute('''
            SELECT * from getCompozit()
            ''')

results = cur.fetchall()
pd.DataFrame(results, columns=['customerid', 'rowguid'])

,customerid,rowguid
0,1,3f5ae95e-b87d-4aed-95b4-c3797afcb74f
1,2,e552f657-a9af-4a7d-a645-c429d6e02491
2,3,130774b1-db21-4ef3-98c8-c104bcd6ed6d
3,4,ff862851-1daa-4044-be7c-3e85583c054d
4,5,83905bdc-6f5e-4f71-b162-c98da069f38a
...,...,...
19815,30114,97154f3d-28f1-4b15-ae03-9518b781ece3
19816,30115,e4cf8fd5-30a4-4b8e-8fd8-47032e255778
19817,30116,ec409609-d25d-41b8-9d15-a1aa6e89fc77
19818,30117,6f08e2fb-1cd3-4f6e-a2e6-385669598b19


In [12]:
conn.rollback()
conn.close()